# **Importar bibliotecas**

In [75]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

# **Fazer o request**  

In [2]:
url = "https://www.ibge.gov.br/indicadores.html"
pagina = requests.get(url)

# **Filtrar e selecionar o conteudo das tabelas**

In [3]:
soup = BeautifulSoup(pagina.content, "html.parser")

tabela = soup.find(class_="indicadores-tabela indicadores-economicos")
titulo = soup.find("th", class_="ultimo").text

rows = tabela.find_all(class_="nonsprite")
index = [id.text for id in rows]

conteudo = tabela.find_all(class_=re.compile(r"table\-accordion.+"))

In [73]:
ultimo = ["vazio" if conteudo[n].find(class_="ultimo") is None else conteudo[n].find(class_="ultimo").text for n in list(range(0,20,2))]
anterior = ["vazio" if conteudo[n].find(class_="desktop-tablet-only anterior") is None else conteudo[n].find(class_="desktop-tablet-only anterior").text for n in list(range(0,20,2))]
doze_meses = ["vazio" if conteudo[n].find(class_="desktop-tablet-only dozemeses") is None else conteudo[n].find(class_="desktop-tablet-only dozemeses").text for n in list(range(0,20,2))]
anual = ["vazio" if conteudo[n].find(class_="desktop-tablet-only ano") is None else conteudo[n].find(class_="desktop-tablet-only ano").text for n in list(range(0,20,2))]

# **Limpeza dos dados**

In [134]:
index = [re.sub(r"\n|^\r\n", "", id).strip() for id in index]

In [137]:
dataset = pd.DataFrame({"Último": ultimo, "Anterior": anterior, "Doze meses": doze_meses, "No ano": anual}, index = index).rename_axis(titulo, axis=0)

In [136]:
dataset.replace(regex=r"\n|\r|^\r\n", value="", inplace = True)

# Transformar dados em integer

dataset

,Último,Anterior,Doze meses,No ano
Indicadores econômicos,,,,
IPCA (%),"Último 0,96 jul 2021","Anterior 0,53 jun 2...","12 meses 8,99 ...","No ano 4,76 ..."
INPC (%),"Último 1,02 jul 2021","Anterior 0,60 jun 2...","12 meses 9,85 ...","No ano 5,01 ..."
IPCA-15 (%),"Último 0,72 jul 2021","Anterior 0,83 jun 2...","12 meses 8,59 ...","No ano 4,88 ..."
IPP (%),"Último 1,31 jun 2021","Anterior 0,99 mai 2...","12 meses 36,81 ...","No ano 19,11 ..."
Custo do m² (%),"Último 1,89 jul 2021","Anterior 2,46 jun 2...","12 meses 22,60 ...","No ano 13,49 ..."
Variação do PIB (%),"Último 1,0 1º tri 2...","Anterior -1,1 4º tr...","12 meses -3,8 ...","No ano 1,0 ..."
PIB per capita (R$),"Último 33.593,82 2...","Anterior 31.843,95 ...",12 meses - ...,No ano - ...
Indústria (%),"Último 0,0 jun 2021","Anterior 1,4 mai 2021","12 meses 6,6 ...","No ano 12,9 ..."
Comércio (%),"Último -1,7 jun 2021","Anterior 2,7 mai 2021","12 meses 5,9 ...","No ano 6,7 ..."


# **Exportar arquivo em excel**

In [ ]:
dataset.to_excel()